<a href="https://colab.research.google.com/github/kimo722504/practice_ML/blob/master/mask_rcnn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection api (MASK_RCNN)

colab에서 ipynb 파일을 키고 그대로 따라 하시면 됩니다. 
**colab**을 사용하기 때문에 대부분 편하게 사용할 수 있습니다. Good~!

1. colab 들어감.
1. File > Open notebook > FILENAME.ipynb
1. Runtime > Change Runtime Type > Hardware Accelerator > GPU 클릭
1. 다른 거 할 필요 없이 **셀**만 실행하면 됨.

>  참고 : [영상](https://www.youtube.com/watch?v=Rgpfk6eYxJA&t=803s)을 먼저 한번 보고오시면 따라할때 더욱더 이해가 잘됩니다.   
영상은 단순히 이해용도이며 MASK_RCNN은 밑에 설명되어있습니다.

## MASK_RCNN블로그와 깃허브 소개
이 노트북은 다음 글의 내용을 그대로 따라하며, 경로만 수정하였습니다.  
[블로그](https://medium.com/@vijendra1125/custom-mask-rcnn-using-tensorflow-object-detection-api-101149ce0765)와 [깃허브](https://github.com/vijendra1125/Custom-Mask-RCNN-using-Tensorfow-Object-detection-API)

#Environment Setup

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
%cd /content/models/research/object_detection/
!git clone https://github.com/vijendra1125/Custom-Mask-RCNN-using-Tensorfow-Object-detection-API
!mv Custom-Mask-RCNN-using-Tensorfow-Object-detection-API multiple_object_mask

In [0]:
!cat -n /content/models/research/object_detection/multiple_object_mask/mask_rcnn_inception_v2_coco.config

## 경로 적절히 수정
경로들이 다 drive(구글 드라이브) 로 되어있기때문에 적절히 수정합시다  

> drive/learning/DL/projects 를 models/research로 치환함  
> drive/learning/DL 을 models/research/object_detection/multiple_object_mask로 치환함


In [0]:
!sed -i 's/drive\/learning\/DL\/projects/models\/research/g' /content/models/research/object_detection/multiple_object_mask/mask_rcnn_inception_v2_coco.config

In [0]:
!sed -i 's/drive\/learning\/DL/models\/research\/object_detection\/multiple_object_mask/g' /content/models/research/object_detection/multiple_object_mask/mask_rcnn_inception_v2_coco.config

## record 만들기
우린 커스텀데이터로 해야하니까 직접 레코드를 만드는 작업을 해봅시다

In [0]:
!cp  ./multiple_object_mask/extra/create_mask_rcnn_tf_record.py ../object_detection/dataset_tools/

In [0]:
%cd /content/models/research
!python object_detection/dataset_tools/create_mask_rcnn_tf_record.py \
--data_dir=object_detection/multiple_object_mask/dataset/ \
--annotations_dir=Annotations \
--image_dir=JPEGImages \
--output_dir=object_detection/multiple_object_mask/dataset/train.record \
--label_map_path=object_detection/multiple_object_mask/dataset/label.pbtxt

## coco trained-models 중 하나인 mask_rcnn_inception_v2_coco 파일을 다운로드
[클릭](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)해서 다운받을수있지만 우리는 코드로 받아봅시다.

In [0]:
import requests
filename = "mask_rcnn_inception_v2_coco_2018_01_28.tar.gz"
URL = "http://download.tensorflow.org/models/object_detection/"+filename
try:
  response = requests.get(URL,stream=True)
  print(response.status_code)
  if response.status_code == 200:
      with open("/content/models/research/object_detection/multiple_object_mask/pre_trained_models/"+filename, 'wb') as f:
          f.write(response.content) # 파일저장하기
except Exception as e:
  print(e)

In [0]:
%cd /content/models/research/object_detection/multiple_object_mask/pre_trained_models
!tar -xvzf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
!mv mask_rcnn_inception_v2_coco_2018_01_28/ mask_rcnn_inception_v2_coco/

#Train Model

In [0]:
%cd /content/models/research/
!python object_detection/legacy/train.py \
--train_dir=/content/models/research/object_detection/multiple_object_mask/CP \
--pipeline_config_path=/content/models/research/object_detection/multiple_object_mask/mask_rcnn_inception_v2_coco.config

#Save inference graph
trained_checkpoint_prefix 경로의 model.ckpt-XXXX 만 적절히 수정하면됩니다.!

In [0]:
!ls -al /content/models/research/object_detection/multiple_object_mask/CP

In [0]:
!python object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=/content/models/research/object_detection/multiple_object_mask/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix=/content/models/research/object_detection/multiple_object_mask/CP/model.ckpt-2000 \
--output_directory=/content/models/research/object_detection/multiple_object_mask/IG
!ls -al /content/models/research/object_detection/multiple_object_mask/IG

## pb 파일을 로컬로 다운로드하세요(마우스로~)
끝~!

In [17]:
!ls -al /content/models/research/object_detection/multiple_object_mask/IG

total 105352
drwxr-xr-x 3 root root     4096 Jul 22 23:57 .
drwxr-xr-x 8 root root     4096 Jul 22 23:43 ..
-rw-r--r-- 1 root root       77 Jul 22 23:57 checkpoint
-rw-r--r-- 1 root root 53402554 Jul 22 23:57 frozen_inference_graph.pb
-rw-r--r-- 1 root root 52726588 Jul 22 23:57 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    16112 Jul 22 23:57 model.ckpt.index
-rw-r--r-- 1 root root  1704235 Jul 22 23:57 model.ckpt.meta
-rw-r--r-- 1 root root     4050 Jul 22 23:57 pipeline.config
-rw-r--r-- 1 root root       34 Jul 22 23:42 ReadMe.txt
drwxr-xr-x 3 root root     4096 Jul 22 23:57 saved_model


# 여기서부터 참고사항

## 툴 사용해서 데이터 만들기
[블로그](https://medium.com/@vijendra1125/custom-mask-rcnn-using-tensorflow-object-detection-api-101149ce0765)를 읽어보면 PixelAnnotationTool 툴을 사용해서 마스킹을 했다고합니다.   
object_detection/multiple_object_mask/dataset/ 위치에 있는 데이터만 바꾸고 학습 돌리면 됩니다.

## xml파일 만들지 않아도됨
create_mask_rcnn_tf_record.py 가 바운딩 박스를 찾아주므로 우리는 따로 xml파일을 만들 필요가 없습니다.  

# 하지만!!!  
[same class에 multiple object가 있을경우(???)](https://github.com/vijendra1125/Custom-Mask-RCNN-using-Tensorfow-Object-detection-API#convert-the-data-to-tensorflow-record-format) create_mask_rcnn_tf_record 파일 대신 create_mask_rcnn_tf_record_multi.py 파일을 사용하여야 하며 이경우 바운딩 박스를 찾아주지 않습니다.  
(아마 한 이미지에 여러 클래스를 마스킹 할 경우는 바운딩 박스도 직접 해주어야 한다는 뜻 같습니다.)   
이 경우 직접 [labelImg library](https://pypi.org/project/labelImg/)로 바운딩박스도 쳐주어야 합니다.  